### Training RL Policies using L5Kit Closed-Loop Environment

This notebook describes how to train RL policies for self-driving using our gym-compatible closed-loop environment.

We will be using [Proximal Policy Optimization (PPO)](https://arxiv.org/abs/1707.06347) algorithm as our reinforcement learning algorithm, as it not only demonstrates remarkable performance but it is also empirically easy to tune.

The PPO implementation in this notebook is based on [Stable Baselines3](https://github.com/DLR-RM/stable-baselines3) framework, a popular framework for training RL policies. Note that our environment is also compatible with [RLlib](https://docs.ray.io/en/latest/rllib.html), another popular frameworks for the same.

ref: 
([rllib] Best workflow to train, save, and test agent #9123
)[https://github.com/ray-project/ray/issues/9123]

In [3]:
import os
os.environ["L5KIT_DATA_FOLDER"] = '/workspace/datasets'
# os.environ["TUNE_RESULT_DIR"] =  '/DATA/l5kit/rllib_tb_logs'

In [4]:
import gym

# from stable_baselines3 import PPO
# from stable_baselines3.common.callbacks import CheckpointCallback
# from stable_baselines3.common.env_util import make_vec_env
# from stable_baselines3.common.utils import get_linear_fn
# from stable_baselines3.common.vec_env import SubprocVecEnv

from l5kit.configs import load_config_data
# from l5kit.environment.feature_extractor import CustomFeatureExtractor
# from l5kit.environment.callbacks import L5KitEvalCallback
from l5kit.environment.envs.l5_env import SimulationConfigGym, GymStepOutput, L5Env

from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet, CLEMetricSet
from prettytable import PrettyTable
import datetime
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
import torch.nn as nn
import numpy as np
import gym
from typing import Dict
import numpy as np
import ray
import pytz


ModuleNotFoundError: No module named 'l5kit.configs'

In [5]:
import l5kit

ModuleNotFoundError: No module named 'l5kit'

## Init ray and env

In [3]:
ray.init(num_cpus=3, ignore_reinit_error=True, log_to_driver=False)

2023-03-16 17:21:30,499	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.7.11
Ray version:,2.2.0


In [3]:
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable
from l5kit.configs import load_config_data

# get environment config
l5kit_path = source/examples/code/config
env_config_path = '/DATA/l5kit/configs/gym_config.yaml'
cfg = load_config_data(env_config_path)


## Customize my model

In [4]:
from ray.rllib.models import ModelCatalog
class GNCNN(TorchModelV2, nn.Module):
    """
    Simple Convolution agent that calculates the required linear output layer
    """

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        super().__init__(obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)

        # raise ValueError(obs_space.shape)
        self._num_objects = obs_space.shape[2] # num_of_channels of input, size x size x channels
        self._num_actions = num_outputs
        self._feature_dim = model_config["custom_model_config"]['feature_dim']

        self.network = nn.Sequential(
            nn.Conv2d(self._num_objects, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(4, 64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(2, 32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=1568, out_features=self._feature_dim),
        )

        self._actor_head = nn.Sequential(
            nn.Linear(self._feature_dim, 256),
            nn.ReLU(),
            nn.Linear(256, self._num_actions),
        )

        self._critic_head = nn.Sequential(
            nn.Linear(self._feature_dim, 1),
        )

    def forward(self, input_dict, state, seq_lens):
        obs_transformed = input_dict['obs'].permute(0, 3, 1, 2) # 32 x 112 x 112 x 7 [B, size, size, channels]
        network_output = self.network(obs_transformed)
        value = self._critic_head(network_output)
        self._value = value.reshape(-1)
        logits = self._actor_head(network_output)
        return logits, state

    def value_function(self):
        return self._value

ModelCatalog.register_custom_model("GN_CNN_torch_model", GNCNN)


In [5]:
from ray.rllib.models import ModelCatalog
class GNCNN_separated(TorchModelV2, nn.Module):
    """
    Simple Convolution agent that calculates the required linear output layer
    """

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        super().__init__(obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)

        # raise ValueError(obs_space.shape)
        self._num_objects = obs_space.shape[2] # num_of_channels of input, size x size x channels
        self._num_actions = num_outputs
        self._feature_dim = model_config["custom_model_config"]['feature_dim']

        self._actor_head = nn.Sequential(
            nn.Conv2d(self._num_objects, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(4, 64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(2, 32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=1568, out_features=self._feature_dim),
            nn.Linear(self._feature_dim, 256),
            nn.ReLU(),
            nn.Linear(256, self._num_actions),
        )

        self._critic_head = nn.Sequential(
            nn.Conv2d(self._num_objects, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(4, 64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(2, 32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=1568, out_features=self._feature_dim),
            nn.Linear(self._feature_dim, 1),
        )

    def forward(self, input_dict, state, seq_lens):
        obs_transformed = input_dict['obs'].permute(0, 3, 1, 2) # 32 x 112 x 112 x 7 [B, size, size, channels]
        # network_output = self.network(obs_transformed)
        value = self._critic_head(obs_transformed)
        self._value = value.reshape(-1)
        logits = self._actor_head(obs_transformed)
        return logits, state

    def value_function(self):
        return self._value
ModelCatalog.register_custom_model(
        "GN_CNN_separated_torch_model", GNCNN_separated
    )


## Define Training and Evaluation Environments

**Training**: We will be training the PPO policy on episodes of length 32 time-steps. We will have 4 sub-processes (training environments) that will help to parallelize and speeden up episode rollouts. The *SimConfig* dataclass will define the parameters of the episode rollout: like length of episode rollout, whether to use log-replayed agents or simulated agents etc.

**Evaluation**: We will evaluate the performance of the PPO policy on the *entire* scene (~248 time-steps).

## Customize gym env

In [6]:
class L5EnvWrapper(gym.Wrapper):
    def __init__(self, env, raster_size = 112, n_channels = 7):
        super().__init__(env)
        self.env = env
        self.n_channels = n_channels
        self.raster_size = raster_size
        obs_shape = (self.raster_size, self.raster_size, self.n_channels)
        self.observation_space =gym.spaces.Box(low=0, high=1, shape=obs_shape, dtype=np.float32)

    def step(self, action:  np.ndarray) -> GymStepOutput:
        # return GymStepOutput(obs, reward["total"], done, info)
        output =  self.env.step(action)
        onlyImageState = output.obs['image'].reshape(self.raster_size, self.raster_size, self.n_channels)
        return GymStepOutput(onlyImageState, output.reward, output.done, output.info)

    def reset(self) -> Dict[str, np.ndarray]:
        return self.env.reset()['image'].reshape(self.raster_size, self.raster_size, self.n_channels)

In [8]:
from ray import tune
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1


# Register , how your env should be constructed (always with 5, or you can take values from the `config` EnvContext object):
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}

tune.register_env("L5-CLE-V0", lambda config: L5Env(**env_kwargs))
tune.register_env("L5-CLE-V1", lambda config: L5EnvWrapper(env = L5Env(**env_kwargs), \
                                                           raster_size= cfg['raster_params']['raster_size'][0], \
                                                           n_channels = 7))

## Train

### ref

Resume stop tune: https://docs.ray.io/en/latest/tune/tutorials/tune-stopping.html

tune.Tuner analysis: https://docs.ray.io/en/latest/rllib/rllib-training.html#basic-python-api

get best result, load from dir: https://docs.ray.io/en/master/tune/examples/tune_analyze_results.html#trial-level-analysis-working-with-an-individual-result

### Hyperparam
```
lr = 3e-4
num_rollout_steps = 256
gamma = 0.8
gae_lambda = 0.9
n_epochs = 10
seed = 42
batch_size = 64
```
### Clipping schedule of PPO epsilon parameter
```
start_val = 0.1
end_val = 0.01
training_progress_ratio = 1.0
clip_schedule = get_linear_fn(start_val, end_val, training_progress_ratio)
```

In [ ]:
!wandb login 083592c84134c040dcca598c644c348d32540a08

In [ ]:
import numpy as np

import ray
from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback
os.environ['WANDB_NOTEBOOK_NAME'] = '/DATA/rllib_ppo_policy_training.ipynb'
os.environ["WANDB_API_KEY"] = '083592c84134c040dcca598c644c348d32540a08'

In [ ]:
import wandb
wandb.init(project="l5kit2", reinit = True)

### shared polocy

In [ ]:
import ray
from ray import air, tune

train_envs = 4
hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")
ray_result_logdir = '/DATA/l5kit/ray_results/' + date

config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    'num_sgd_iter': 5,
    'sgd_minibatch_size': 256,
    # 'num_cpus_per_worker': 0,  # This avoids running out of resources in the notebook environment when this cell is re-executed
    "model": {
        "custom_model": "GN_CNN_torch_model",
        "custom_model_config": {'feature_dim':128},
    },
    
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
    "lr": 0.0003,
    "gamma": 0.8,
    'seed': 42,
}

result_grid = tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(6e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=4, checkpoint_frequency = 10, checkpoint_score_attribute = 'episode_reward_mean')
        ),
    param_space=config_param_space).fit()

### Restore a specific checkpoint

In [ ]:
import ray
from ray import air, tune
hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")
ray_result_logdir = '/DATA/l5kit/ray_results/' + date

train_envs = 4
lr = 3e-3
lr_start = 3e-4
lr_end = 3e-5
lr_time = int(4e6)

config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    "model": {
        "custom_model": "GN_CNN_torch_model",
        "custom_model_config": {'feature_dim':128},
    },
    
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
    "lr": lr,
    'seed': 42,
    "lr_schedule": [
        [1e6, lr_start],
        [2e6, lr_end],
    ],
    'train_batch_size': 8000,
    'sgd_minibatch_size': 1024,
    'num_sgd_iter': 16,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 32,
    'gamma': 0.8,    
}

result_grid = tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(6e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=4, 
                                               checkpoint_frequency = 10, 
                                               checkpoint_score_attribute = 'episode_reward_mean'),
        callbacks=[WandbLoggerCallback(project="l5kit2", save_checkpoints=True),],
        ),
    param_space=config_param_space).fit()
    

In [ ]:
config_param_space['lr'] = 3e-5
path_to_trained_agent_checkpoint = 'l5kit/ray_results/27-12-2022_16-24-37/PPO/PPO_L5-CLE-V1_fdd0f_00000_0_2022-12-27_11-45-42/checkpoint_000480'
from ray.rllib.algorithms.ppo import PPO
ray.tune.run(PPO, config=config_param_space, restore=path_to_trained_agent_checkpoint)

In [ ]:
import ray
from ray import air, tune
ray_result_logdir = '/DATA/l5kit/ray_results'
config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    'train_batch_size': 4000,
    'sgd_minibatch_size': 4096,
    "num_envs_per_worker": 32,
    'num_sgd_iter': 32,
    'sgd_minibatch_size': 256,
    # 'num_cpus_per_worker': 0,  # This avoids running out of resources in the notebook environment when this cell is re-executed
    "model": {
        "custom_model": "GN_CNN_torch_model",
        "custom_model_config": {'feature_dim':128},
    },
    
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     # "log_level": "DEBUG",
     "restart_failed_sub_environments": True,
    "lr": 0.0003,
    "gamma": 0.8,
    'seed': 42,
}

result_grid = tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": -30, 'timesteps_total': int(12e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=4, checkpoint_frequency = 10, checkpoint_score_attribute = 'episode_reward_mean')
        ),
    param_space=config_param_space).fit()

In [ ]:
ray_result_logdir = '/DATA/l5kit/ray_results'

tuner = tune.Tuner.restore(
    path=ray_result_logdir + '/PPO',
)
result_grid = tuner.fit()

In [ ]:
result_grid

# Get the result with the maximum test set `mean_accuracy`
best_result = result_grid.get_best_result()
best_result.checkpoint

In [ ]:
best_result.metrics

In [ ]:
result_df = best_result.metrics_dataframe
result_df[['num_agent_steps_sampled','episode_reward_mean'] ]

In [ ]:
best_result.metrics_dataframe.plot("num_agent_steps_sampled", "episode_reward_mean")


In [ ]:
for result in result_grid:
    print(1)

### Separated policy

In [ ]:
import ray
from ray import air, tune
train_envs = 4
ray_result_logdir = '/DATA/l5kit/ray_results'
lr = 3e-3
lr_start = 3e-4
lr_end = 3e-5
lr_time = int(4e6)

config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    '_disable_preprocessor_api': True,
    "eager_tracing": True,
    "restart_failed_sub_environments": True,
    "lr": lr_start,
    # "model": {
    #     "custom_model": "GN_CNN_separated_torch_model",
    #     "custom_model_config": {'feature_dim':128},
    # },
    'model' : {
            "dim": 84,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
            "vf_share_layers": False,   
    },
#
    "lr": lr,
    'seed': 42,
    "lr_schedule": [
        [1e6, lr_start],
        [2e6, lr_end],
    ],
    'train_batch_size': 8000,
    'sgd_minibatch_size': 256,
    'num_sgd_iter': 16,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 32,
    'gamma': 0.8,
}

result_grid = tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(6e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=4, checkpoint_frequency = 10, checkpoint_score_attribute = 'episode_reward_mean'),
        callbacks=[WandbLoggerCallback(project="l5kit2", save_checkpoints=True),],
        ),
    param_space=config_param_space).fit()

In [ ]:
ray_result_logdir = '/DATA/l5kit/ray_results'

tuner = tune.Tuner.restore(
    path=ray_result_logdir + '/PPO',
)
tuner.fit()

NOTE: Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`

2022-12-04 05:50:38,570	INFO experiment_analysis.py:795 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.

2022-12-04 05:50:39,684	INFO trial_runner.py:601 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2022-12-04 05:50:39,687	INFO trial_runner.py:738 -- Using following checkpoint to resume: /content/drive/MyDrive/Colab Notebooks/l5kit/ray_results/PPO/experiment_state-2022-12-04_05-28-55.json

2022-12-04 05:50:39,710	WARNING trial_runner.py:743 -- Attempting to resume experiment from /content/drive/MyDrive/Colab Notebooks/l5kit/ray_results/PPO. This will ignore any new changes to the specification.

2022-12-04 05:50:40,703	INFO tune.py:668 -- TrialRunner resumed, ignoring new add_experiment but updating trial resources.

**Voila!** We have a trained PPO policy! Train for larger number of steps for better accuracy. Typical RL algorithms require training atleast 1M steps for good convergence. You can visualize the quantitiative evaluation using tensorboard.

## Visualize the episode from the environment

We can easily visualize the outputs obtained by rolling out episodes in the L5Kit using the Bokeh visualizer.

In [7]:
train_envs = 4
lr = 3e-3
lr_start = 3e-4
lr_end = 3e-5
lr_time = int(4e6)

config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    # "num_workers": 63,
    "num_envs_per_worker": train_envs,
    "model": {
        "custom_model": "GN_CNN_torch_model",
        "custom_model_config": {'feature_dim':128},
    },
    
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
    "lr": lr,
    'seed': 42,
    "lr_schedule": [
        [1e6, lr_start],
        [2e6, lr_end],
    ],
    'train_batch_size': 8000,
    'sgd_minibatch_size': 1024,
    # 'num_sgd_iter': 16,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 32,
    'gamma': 0.8,    
}


In [8]:
from ray import tune
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None

env_kwargs = {'env_config_path': env_config_path, 
              'use_kinematic': True, 
              'sim_cfg': rollout_sim_cfg,  
              'train': False, 
              'return_info': True}

rollout_env = L5EnvWrapper(env = L5Env(**env_kwargs), \
                           raster_size= cfg['raster_params']['raster_size'][0], \
                           n_channels = 7,)
tune.register_env("L5-CLE-V2", 
                  lambda config: L5EnvWrapper(env = L5Env(**env_kwargs), \
                                              raster_size= cfg['raster_params']['raster_size'][0], \
                                              n_channels = 7))

In [26]:
from ray.rllib.algorithms.ppo import PPO
checkpoint_path = 'l5kit/ray_results/31-12-2022_23-45-14/PPO/PPO_L5-CLE-V1_802de_00000_0_2022-12-31_16-45-15/checkpoint_000400'
algo = PPO(config=config_param_space, env='L5-CLE-V2')
algo.restore(checkpoint_path)

2023-01-11 03:52:52,652	WARNING deprecation.py:47 -- DeprecationWarning: `algo = Algorithm(env='L5-CLE-V2', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('L5-CLE-V2').build()` instead. This will raise an error in the future!
(RolloutWorker pid=4156390) /usr/local/lib/python3.8/dist-packages/l5kit/data/zarr_dataset.py:213: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
(RolloutWorker pid=4156390)   dataset = ChunkedDataset("")
(RolloutWorker pid=4156390) /usr/local/lib/python3.8/dist-packages/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
(RolloutWorker pid=4156390)   new_dataset = ChunkedDataset("")
(RolloutWorker pid=4156389) 2023-01-11 03:53:00,462	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you

In [27]:
rewards = []
def rollout_episode(model, env, idx = 0):
    """Rollout a particular scene index and return the simulation output.

    :param model: the RL policy
    :param env: the gym environment
    :param idx: the scene index to be rolled out
    :return: the episode output of the rolled out scene
    """

    # Set the reset_scene_id to 'idx'
    env.set_reset_id(idx)
    
    # Rollout step-by-step
    obs = env.reset()
    done = False
    total_reward = 0
    while True:
        action = model.compute_single_action(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        if done:
            rewards.append(total_reward)
            break

    # The episode outputs are present in the key "sim_outs"
    sim_out = info["sim_outs"][0]
    return sim_out

# Rollout one episode
# sim_out = rollout_episode(model, rollout_env)
# Rollout 5 episodes
sim_outs =[]
for i in range(100):
    sim_outs.append(rollout_episode(algo, rollout_env, i))

In [23]:
print(rewards)

[-1700.6033111233264, -2718.2281629676, -1925.2783587416634, -1416.929284658283, -2610.9406875574496]


In [ ]:
# might change with different rasterizer
map_API = rollout_env.dataset.rasterizer.sem_rast.mapAPI

def visualize_outputs(sim_outs, map_API):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, map_API)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
visualize_outputs(sim_outs, map_API)

## Calculate the performance metrics from the episode outputs

We can also calculate the various quantitative metrics on the rolled out episode output. 

In [28]:
def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else L2DisplacementYawMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4)])
    print(table)


quantify_outputs(sim_outs)

+----------+-----------+----------+
| scene_id |    FDE    |   ADE    |
+----------+-----------+----------+
|    0     |  123.2568 | 52.4588  |
|    1     |  116.6584 | 22.8952  |
|    2     |  52.0705  | 30.8244  |
|    3     |  132.0699 | 44.7191  |
|    4     |  144.1401 | 48.6937  |
|    5     |  64.7322  | 30.8698  |
|    6     |  230.0992 | 113.1657 |
|    7     |  22.4712  | 10.1924  |
|    8     |  477.5516 | 152.0144 |
|    9     |  241.9794 | 89.2365  |
|    10    |  144.5365 | 55.7827  |
|    11    |  73.0268  | 34.0385  |
|    12    |  18.6581  |  11.226  |
|    13    |  52.8625  | 19.2529  |
|    14    | 1748.6797 | 464.7148 |
|    15    |   47.387  | 31.2623  |
|    16    |   2.6316  |  5.2471  |
|    17    |  113.6218 | 37.0894  |
|    18    |  313.8562 | 136.6049 |
|    19    |   5.5315  |  4.0874  |
|    20    |  35.7629  |  7.0497  |
|    21    |  17.9055  |  9.1302  |
|    22    |  51.5885  | 19.6745  |
|    23    |  26.9764  |  5.9624  |
|    24    |  495.1066 | 170

In [12]:
def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else CLEMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE", "DRT", "CF", "CR", "CS", "PEGO"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    tot_drt = 0.0
    tot_cf = 0.0
    tot_cr = 0.0
    tot_cs = 0.0
    tot_p_ego = 0.0
    tot_a_ego = 0.0
    # print(scene_results[0])
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        drt_error = scene_metrics['distance_to_reference_trajectory'][-1]
        cf_error = scene_metrics['collision_front'][-1]
        cr_error = scene_metrics['collision_rear'][-1]
        cs_error = scene_metrics['collision_side'][-1]
        p_ego = scene_metrics['simulated_minus_recorded_ego_speed'][-1]
        # a_ego = scene_metrics['aggressive_ego'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4), round(drt_error.item(), 4), round(cf_error.item(), 4), round(cr_error.item(), 4), 
        round(cs_error.item(), 4), round(p_ego.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()
        tot_drt += drt_error.item()
        tot_cf += cf_error.item()
        tot_cr += cr_error.item()
        tot_cs += cs_error.item()
        tot_p_ego += p_ego.item()
        # tot_a_ego += a_ego.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    ave_drt = tot_drt / len(scene_results)
    ave_cf = tot_cf / len(scene_results)
    ave_cr = tot_cr / len(scene_results)
    ave_cs = tot_cs / len(scene_results)
    ave_p_ego = tot_p_ego / len(scene_results)
    # ave_a_ego = tot_a_ego / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4), round(ave_drt, 4), round(ave_cf, 4), round(ave_cr, 4), round(ave_cs, 4), round(ave_p_ego, 4)])
    print(table)


quantify_outputs(sim_outs)

/usr/local/lib/python3.8/dist-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


+----------+-----------+----------+----------+------+-----+------+----------+
| scene_id |    FDE    |   ADE    |   DRT    |  CF  |  CR |  CS  |   PEGO   |
+----------+-----------+----------+----------+------+-----+------+----------+
|    0     |  182.7081 | 70.5777  |  0.4223  | 0.0  | 0.0 | 0.0  | -13.3355 |
|    1     |  22.3221  |  8.4134  | 18.1887  | 0.0  | 0.0 | 0.0  | 20.5099  |
|    2     |  126.4999 | 41.8324  | 81.7555  | 0.0  | 0.0 | 0.0  |  8.3309  |
|    3     |  17.8828  | 25.7624  | 42.8494  | 0.0  | 0.0 | 0.0  |  5.565   |
|    4     |  57.8033  | 36.3409  | 66.1168  | 0.0  | 0.0 | 0.0  |  2.9566  |
|    5     |  102.8478 | 48.3509  | 84.7749  | 0.0  | 0.0 | 0.0  |  9.1701  |
|    6     |  299.2674 | 136.3642 | 52.9568  | 0.0  | 0.0 | 0.0  | -9.7238  |
|    7     |  43.3553  | 18.9104  |  0.5098  | 0.0  | 0.0 | 0.0  |  0.0387  |
|    8     |  411.7862 | 114.802  | 52.6135  | 0.0  | 0.0 | 0.0  |  8.8057  |
|    9     |  200.6255 | 76.5964  |  0.0872  | 0.0  | 0.0 | 0.0 